In [53]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

### Example 1

Maximize z = 5x + 4y

s.t.

    6x+4y <= 24
    1x+2y <= 6
    -1x+1y <= 1

In [58]:
model = pyo.ConcreteModel()

model.x = pyo.Var(bounds=(0, None))
model.y = pyo.Var(bounds=(0, 2))

x = model.x
y = model.y

model.obj = pyo.Objective(expr= 5*x+4*y, sense=maximize)
model.c1 = pyo.Constraint(expr= 6*x+4*y <= 24)
model.c2 = pyo.Constraint(expr= 1*x+2*y <= 6)
model.c3 = pyo.Constraint(expr= -1*x+1*y <= 1)

opt = SolverFactory('glpk')
opt.solve(model)

x_value = pyo.value(x)
y_value = pyo.value(y)
z_value = model.obj()
print('x:', x_value)
print('y:', y_value)
print('z:', z_value)

model.pprint()

x: 3.0
y: 1.5
z: 21.0
2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   3.0 :  None : False : False :  Reals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.5 :     2 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 5*x + 4*y

3 Constraint Declarations
    c1 : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None :  -Inf : 6*x + 4*y :  24.0 :   True
    c2 : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : x + 2*y :   6.0 :   True
    c3 : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : - x + y :   1.0 :   True

6 Declarations: x y obj c1 c2 c3


### Transportation Model

In [20]:
with open("input_transp.json", mode="r", encoding='utf8') as file:
    input_data = json.load(file)
input_data

{'availabilities': {'S1': 14, 'S2': 26, 'S3': 11},
 'demands': {'C1': 5, 'C2': 13, 'C3': 15, 'C4': 17},
 'costs': [{'from': 'S1', 'to': 'C1', 'value': 10},
  {'from': 'S1', 'to': 'C2', 'value': 5},
  {'from': 'S1', 'to': 'C3', 'value': 20},
  {'from': 'S1', 'to': 'C4', 'value': 12},
  {'from': 'S2', 'to': 'C1', 'value': 12},
  {'from': 'S2', 'to': 'C2', 'value': 7},
  {'from': 'S2', 'to': 'C3', 'value': 12},
  {'from': 'S2', 'to': 'C4', 'value': 19},
  {'from': 'S3', 'to': 'C1', 'value': 6},
  {'from': 'S3', 'to': 'C2', 'value': 12},
  {'from': 'S3', 'to': 'C3', 'value': 16},
  {'from': 'S3', 'to': 'C4', 'value': 17}]}

In [30]:
#create local variable to store input parameters
availabilities = input_data['availabilities']
demands = input_data['demands']

#dictionary of costs indexed by tuples (origin, destination)
costs = {(c['from'], c['to']): c['value'] for c in input_data['costs']}
costs

{('S1', 'C1'): 10,
 ('S1', 'C2'): 5,
 ('S1', 'C3'): 20,
 ('S1', 'C4'): 12,
 ('S2', 'C1'): 12,
 ('S2', 'C2'): 7,
 ('S2', 'C3'): 12,
 ('S2', 'C4'): 19,
 ('S3', 'C1'): 6,
 ('S3', 'C2'): 12,
 ('S3', 'C3'): 16,
 ('S3', 'C4'): 17}

In [31]:
availabilities.keys(), demands.keys()

(dict_keys(['S1', 'S2', 'S3']), dict_keys(['C1', 'C2', 'C3', 'C4']))

In [32]:
model = pyo.ConcreteModel()
# suppliers I and customers J
model.I = pyo.Set(initialize=availabilities.keys())
model.J = pyo.Set(initialize=demands.keys())
model.I.display()

I : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    3 : {'S1', 'S2', 'S3'}


In [33]:
# parameters
model.b = pyo.Param(model.I, initialize=availabilities)
model.d = pyo.Param(model.J, initialize=demands)
model.c = pyo.Param(model.I, model.J, initialize=costs)
model.c.display()

c : Size=12, Index=I*J, Domain=Any, Default=None, Mutable=False
    Key          : Value
    ('S1', 'C1') :    10
    ('S1', 'C2') :     5
    ('S1', 'C3') :    20
    ('S1', 'C4') :    12
    ('S2', 'C1') :    12
    ('S2', 'C2') :     7
    ('S2', 'C3') :    12
    ('S2', 'C4') :    19
    ('S3', 'C1') :     6
    ('S3', 'C2') :    12
    ('S3', 'C3') :    16
    ('S3', 'C4') :    17


In [38]:
# dicision variables
model.x = pyo.Var(model.I, model.J, within=pyo.NonNegativeReals)
model.x.display()

'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
x : Size=12, Index=I*J
    Key          : Lower : Value : Upper : Fixed : Stale : Domain
    ('S1', 'C1') :     0 :  None :  None : False :  True : NonNegativeReals
    ('S1', 'C2') :     0 :  None :  None : False :  True : NonNegativeReals
    ('S1', 'C3') :     0 :  None :  None : False :  True : NonNegativeReals
    ('S1', 'C4') :     0 :  None :  None : False :  True : NonNegativeReals
    ('S2', 'C1') :     0 :  None :  None : False :  True : NonNegativeReals
    ('S2', 'C2') :     0 :  None :  None : False :  True : NonNegativeReals
    ('S2', 'C3') :     0 :  None :  None : False :  True : NonNegativeReals
    ('S2', 'C4') :     0 :  None :  None : False :  True : NonNegativeReals
    ('S3', 'C1') :     0 :  None :  None : False :  True

In [39]:
#supplier availability constraints
def av_cstr(model, i):
    return sum(model.x[i, :]) <= model.b[i]
model.av_cstr = pyo.Constraint(model.I, rule=av_cstr)
model.av_cstr.pprint()

av_cstr : Size=3, Index=I, Active=True
    Key : Lower : Body                                      : Upper : Active
     S1 :  -Inf : x[S1,C1] + x[S1,C2] + x[S1,C3] + x[S1,C4] :  14.0 :   True
     S2 :  -Inf : x[S2,C1] + x[S2,C2] + x[S2,C3] + x[S2,C4] :  26.0 :   True
     S3 :  -Inf : x[S3,C1] + x[S3,C2] + x[S3,C3] + x[S3,C4] :  11.0 :   True


In [44]:
#demand equality constraints
def dem_cstr(model, j):
    return sum(model.x[:, j]) == model.d[j]
model.dem_cstr = pyo.Constraint(model.J, rule=dem_cstr)
model.dem_cstr.pprint()

'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>). This
is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
dem_cstr : Size=4, Index=J, Active=True
    Key : Lower : Body                           : Upper : Active
     C1 :   5.0 : x[S1,C1] + x[S2,C1] + x[S3,C1] :   5.0 :   True
     C2 :  13.0 : x[S1,C2] + x[S2,C2] + x[S3,C2] :  13.0 :   True
     C3 :  15.0 : x[S1,C3] + x[S2,C3] + x[S3,C3] :  15.0 :   True
     C4 :  17.0 : x[S1,C4] + x[S2,C4] + x[S3,C4] :  17.0 :   True


In [47]:
#objective function
def obj(model):
    total_cost = sum(model.c[i, j]*model.x[i, j] for i in model.I for j in model.J)
    return total_cost
model.obj = pyo.Objective(rule=obj, sense=pyo.minimize)

'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [48]:
solver = pyo.SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 526.0, 'Upper bound': 526.0, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 12, 'Number of nonzeros': 24, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.046597957611083984}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [49]:
model.obj()

526.0

In [50]:
model.x.extract_values()

{('S1', 'C1'): 0.0,
 ('S1', 'C2'): 2.0,
 ('S1', 'C3'): 0.0,
 ('S1', 'C4'): 12.0,
 ('S2', 'C1'): 0.0,
 ('S2', 'C2'): 11.0,
 ('S2', 'C3'): 15.0,
 ('S2', 'C4'): 0.0,
 ('S3', 'C1'): 5.0,
 ('S3', 'C2'): 0.0,
 ('S3', 'C3'): 0.0,
 ('S3', 'C4'): 5.0}

In [51]:
sol = [
    {'from': i, 'to': j, 'value': val}
    for (i, j), val in model.x.extract_values().items()
]
sol

[{'from': 'S1', 'to': 'C1', 'value': 0.0},
 {'from': 'S1', 'to': 'C2', 'value': 2.0},
 {'from': 'S1', 'to': 'C3', 'value': 0.0},
 {'from': 'S1', 'to': 'C4', 'value': 12.0},
 {'from': 'S2', 'to': 'C1', 'value': 0.0},
 {'from': 'S2', 'to': 'C2', 'value': 11.0},
 {'from': 'S2', 'to': 'C3', 'value': 15.0},
 {'from': 'S2', 'to': 'C4', 'value': 0.0},
 {'from': 'S3', 'to': 'C1', 'value': 5.0},
 {'from': 'S3', 'to': 'C2', 'value': 0.0},
 {'from': 'S3', 'to': 'C3', 'value': 0.0},
 {'from': 'S3', 'to': 'C4', 'value': 5.0}]

In [55]:
df = pd.DataFrame(sol).pivot(index='from', columns='to', values='value')
df

to,C1,C2,C3,C4
from,,,,
S1,0.0,2.0,0.0,12.0
S2,0.0,11.0,15.0,0.0
S3,5.0,0.0,0.0,5.0
